<a href="https://colab.research.google.com/github/HardikxLabeitazer/DataAnalysis/blob/master/DataAnalysis6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
weather = pd.read_csv('/content/drive/MyDrive/DataAnalysis/Ch4/nyc_weather_2018.csv',parse_dates=['date'])

In [3]:
fb = pd.read_csv('/content/drive/MyDrive/DataAnalysis/Ch4/fb_2018.csv',parse_dates=True)

In [6]:
#Using DataFrame Operation To Enrich Data
#Arithmetic And Statistics
fb.assign(abs_z_score_volume = lambda x:x.volume.sub(x.volume.mean()).div(x.volume.std()).abs()).query('abs_z_score_volume > 3')

,date,open,high,low,close,volume,abs_z_score_volume
52,2018-03-19,177.01,177.17,170.06,172.56,88140060,3.145078
53,2018-03-20,167.47,170.20,161.95,168.15,129851768,5.315169
54,2018-03-21,164.80,173.40,163.30,169.39,106598834,4.105413
57,2018-03-26,160.82,161.10,149.02,160.06,126116634,5.120845
142,2018-07-26,174.89,180.13,173.75,176.26,169803668,7.393705


In [8]:
fb.assign(
    volume_pct_change = fb.volume.pct_change(),
    pct_change_rank = lambda x:x.volume_pct_change.abs().rank(ascending=False)
).nsmallest(5,'pct_change_rank')

,date,open,high,low,close,volume,volume_pct_change,pct_change_rank
8,2018-01-12,178.06,181.48,177.40,179.37,77551299,7.087876,1.0
52,2018-03-19,177.01,177.17,170.06,172.56,88140060,2.611789,2.0
142,2018-07-26,174.89,180.13,173.75,176.26,169803668,1.628841,3.0
182,2018-09-21,166.64,167.25,162.81,162.93,45994800,1.428956,4.0
57,2018-03-26,160.82,161.10,149.02,160.06,126116634,1.352496,5.0


In [16]:
fb['2018-01-11':'2018-01-12']

,open,high,low,close,volume
date,,,,,
2018-01-11,188.40,188.40,187.38,187.77,9588587
2018-01-12,178.06,181.48,177.40,179.37,77551299


In [17]:
(fb>215).any()

open       True
high       True
low       False
close      True
volume     True
dtype: bool

In [18]:
(fb>215).all()

open      False
high      False
low       False
close     False
volume     True
dtype: bool

In [19]:
#Binning
(fb.volume.value_counts()>1).sum()

0

In [20]:
volume_binned= pd.cut(
    fb.volume,bins=3,labels=['low','med','high']
)
volume_binned.value_counts()

low     240
med       8
high      3
Name: volume, dtype: int64

In [23]:
fb[volume_binned=='high'].sort_values('volume',ascending=False)

,open,high,low,close,volume
date,,,,,
2018-07-26,174.89,180.13,173.75,176.26,169803668
2018-03-20,167.47,170.20,161.95,168.15,129851768
2018-03-26,160.82,161.10,149.02,160.06,126116634


In [24]:
fb['2018-07-25':'2018-07-26']

,open,high,low,close,volume
date,,,,,
2018-07-25,215.715,218.62,214.27,217.50,64592585
2018-07-26,174.890,180.13,173.75,176.26,169803668


In [25]:
fb['2018-03-16':'2018-03-20']

,open,high,low,close,volume
date,,,,,
2018-03-16,184.49,185.33,183.41,185.09,24403438
2018-03-19,177.01,177.17,170.06,172.56,88140060
2018-03-20,167.47,170.20,161.95,168.15,129851768


In [26]:
volume_qbinned= pd.qcut(
    fb.volume,q=4,labels=['q1','q2','q3','q4']
)
volume_qbinned.value_counts()

q1    63
q2    63
q4    63
q3    62
Name: volume, dtype: int64

In [27]:
#Applying Functions
central_park_weather = weather.query(
    'station=="GHCND:USW00094728" '
).pivot(index='date',columns='datatype',values='value')

In [28]:
oct_weather_z_scores = central_park_weather.loc['2018-10',['TMIN','TMAX','PRCP']].apply(lambda x:x.sub(x.mean()).div(x.std()))

In [29]:
oct_weather_z_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
datatype,,,,,,,,
TMIN,31.0,-1.790682e-16,1.0,-1.339112,-0.751019,-0.474269,1.065152,1.843511
TMAX,31.0,-3.010585e-16,1.0,-1.305582,-0.870013,-0.138258,1.011643,1.604016
PRCP,31.0,6.088320e-17,1.0,-0.394438,-0.394438,-0.394438,-0.240253,3.936167


In [30]:
oct_weather_z_scores.query('PRCP > 3').PRCP

date
2018-10-27    3.936167
Name: PRCP, dtype: float64

In [31]:
central_park_weather.loc['2018-10','PRCP'].describe()

count    31.000000
mean      2.941935
std       7.458542
min       0.000000
25%       0.000000
50%       0.000000
75%       1.150000
max      32.300000
Name: PRCP, dtype: float64

In [32]:
#Windows Calculations
#Rolling Windows
central_park_weather.loc['2018-10'].assign(
    rolling_PRCP = lambda x:x.PRCP.rolling('3D').sum()
)[['PRCP','rolling_PRCP']].head(7).T

date,2018-10-01,2018-10-02,2018-10-03,2018-10-04,2018-10-05,2018-10-06,2018-10-07
datatype,,,,,,,
PRCP,0.0,17.5,0.0,1.0,0.0,0.0,0.0
rolling_PRCP,0.0,17.5,17.5,18.5,1.0,1.0,0.0


In [33]:
central_park_weather.loc['2018-10'].rolling('3D').mean().head(7).iloc[:,:6]

datatype,AWND,PRCP,SNOW,SNWD,TMAX,TMIN
date,,,,,,
2018-10-01,0.900000,0.000000,0.0,0.0,24.400000,17.200000
2018-10-02,0.900000,8.750000,0.0,0.0,24.700000,17.750000
2018-10-03,0.966667,5.833333,0.0,0.0,24.233333,17.566667
2018-10-04,0.800000,6.166667,0.0,0.0,24.233333,17.200000
2018-10-05,1.033333,0.333333,0.0,0.0,23.133333,16.300000
2018-10-06,0.833333,0.333333,0.0,0.0,22.033333,16.300000
2018-10-07,1.066667,0.000000,0.0,0.0,22.600000,17.400000


In [34]:
central_park_weather['2018-10-01':'2018-10-07'].rolling('3D').agg({
    'TMAX':'max','TMIN':'min',
    'AWND':'mean','PRCP':'sum'
}).join(
    central_park_weather[['TMAX','TMIN','AWND','PRCP']],lsuffix='_rolling'
).sort_index(axis=1)

datatype,AWND,AWND_rolling,PRCP,PRCP_rolling,TMAX,TMAX_rolling,TMIN,TMIN_rolling
date,,,,,,,,
2018-10-01,0.9,0.900000,0.0,0.0,24.4,24.4,17.2,17.2
2018-10-02,0.9,0.900000,17.5,17.5,25.0,25.0,18.3,17.2
2018-10-03,1.1,0.966667,0.0,17.5,23.3,25.0,17.2,17.2
2018-10-04,0.4,0.800000,1.0,18.5,24.4,25.0,16.1,16.1
2018-10-05,1.6,1.033333,0.0,1.0,21.7,24.4,15.6,15.6
2018-10-06,0.5,0.833333,0.0,1.0,20.0,24.4,17.2,15.6
2018-10-07,1.1,1.066667,0.0,0.0,26.1,26.1,19.4,15.6


In [35]:
#Expanding window
central_park_weather.loc['2018-06'].assign(
    TOTAL_PRCP = lambda x:x.PRCP.cumsum(),
    AVG_PRCP= lambda x:x.PRCP.expanding().mean()
).head(10)[['PRCP','TOTAL_PRCP','AVG_PRCP']].T

date,2018-06-01,2018-06-02,2018-06-03,2018-06-04,2018-06-05,2018-06-06,2018-06-07,2018-06-08,2018-06-09,2018-06-10
datatype,,,,,,,,,,
PRCP,6.9,2.00,6.4,4.10,0.00,0.000000,0.000000,0.000,0.000000,0.30
TOTAL_PRCP,6.9,8.90,15.3,19.40,19.40,19.400000,19.400000,19.400,19.400000,19.70
AVG_PRCP,6.9,4.45,5.1,4.85,3.88,3.233333,2.771429,2.425,2.155556,1.97


In [36]:
central_park_weather['2018-10-01':'2018-10-07'].expanding().agg({
    'TMAX':np.max,'TMIN':np.min,
    'AWND':np.mean,'PRCP':np.sum
}).join(
    central_park_weather[['TMAX','TMIN','AWND','PRCP']],lsuffix='_expanding'
).sort_index(axis=1)

datatype,AWND,AWND_expanding,PRCP,PRCP_expanding,TMAX,TMAX_expanding,TMIN,TMIN_expanding
date,,,,,,,,
2018-10-01,0.9,0.900000,0.0,0.0,24.4,24.4,17.2,17.2
2018-10-02,0.9,0.900000,17.5,17.5,25.0,25.0,18.3,17.2
2018-10-03,1.1,0.966667,0.0,17.5,23.3,25.0,17.2,17.2
2018-10-04,0.4,0.825000,1.0,18.5,24.4,25.0,16.1,16.1
2018-10-05,1.6,0.980000,0.0,18.5,21.7,25.0,15.6,15.6
2018-10-06,0.5,0.900000,0.0,18.5,20.0,25.0,17.2,15.6
2018-10-07,1.1,0.928571,0.0,18.5,26.1,26.1,19.4,15.6


In [37]:
central_park_weather.assign(
    AVG=lambda x:x.TMAX.rolling('30D').mean(),
    EWMA = lambda x:x.TMAX.ewm(span=30).mean()
).loc['2018-09-29':'2018-10-08',['TMAX','EWMA','AVG']].T

date,2018-09-29,2018-09-30,2018-10-01,2018-10-02,2018-10-03,2018-10-04,2018-10-05,2018-10-06,2018-10-07,2018-10-08
datatype,,,,,,,,,,
TMAX,22.200000,21.100000,24.400000,25.000000,23.300000,24.400000,21.700000,20.000000,26.100000,23.300000
EWMA,24.410887,24.197281,24.210360,24.261304,24.199285,24.212234,24.050154,23.788854,23.937960,23.896802
AVG,24.723333,24.573333,24.533333,24.460000,24.163333,23.866667,23.533333,23.070000,23.143333,23.196667


In [40]:
#Pipes
# data.pipe(h).pipe(g,20).pipe(f,x=True)

In [42]:
#Aggregating data
fb = pd.read_csv('/content/drive/MyDrive/DataAnalysis/Ch4/fb_2018.csv',index_col='date',parse_dates=True,).assign(
    trading_volume=lambda x:pd.cut(x.volume,bins=3,labels=['low','med','high'])
)

weather =pd.read_csv('/content/drive/MyDrive/DataAnalysis/Ch4/weather_by_station.csv',index_col='date',parse_dates=True)

In [43]:
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low
2018-01-08,187.20,188.90,186.3300,188.28,17994726,low


In [44]:
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"


In [45]:
pd.set_option('display.float_format',lambda x:'%.2f' % x)

In [46]:
#Summarizing DataFrames
fb.agg({
    'open':np.mean,'high':np.max,'low':np.min,
    'close':np.mean, 'volume':np.sum
})

open            171.45
high            218.62
low             123.02
close           171.51
volume   6949682394.00
dtype: float64

In [49]:
weather.query('station=="GHCND:USW00094728"').pivot(columns='datatype',values='value')[['SNOW','PRCP']].sum()

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

In [50]:
fb.agg({
    'open':'mean',
    'high':['min','max'],
    'low':['min','max'],
    'close':'mean'
})

,open,high,low,close
mean,171.45,NaN,NaN,171.51
min,NaN,129.74,123.02,NaN
max,NaN,218.62,214.27,NaN


In [51]:
fb.groupby('trading_volume').mean()

,open,high,low,close,volume
trading_volume,,,,,
low,171.36,173.46,169.31,171.43,24547207.71
med,175.82,179.42,172.11,175.14,79072559.12
high,167.73,170.48,161.57,168.16,141924023.33


In [52]:
fb.groupby('trading_volume')['close'].agg(['min','max','mean'])

,min,max,mean
trading_volume,,,
low,124.06,214.67,171.43
med,152.22,217.50,175.14
high,160.06,176.26,168.16


In [53]:
fb_agg = fb.groupby('trading_volume').agg({
    'open':'mean','high':['min','max'],
    'low':['min','max'],'close':'mean'
})
fb_agg

open   high           low         close
                 mean    min    max    min    max   mean
trading_volume                                          
low            171.36 129.74 216.20 123.02 212.60 171.43
med            175.82 162.85 218.62 150.75 214.27 175.14
high           167.73 161.10 180.13 149.02 173.75 168.16

In [54]:
fb_agg.columns

MultiIndex([( 'open', 'mean'),
            ( 'high',  'min'),
            ( 'high',  'max'),
            (  'low',  'min'),
            (  'low',  'max'),
            ('close', 'mean')],
           )

In [56]:
fb_agg.columns=['_'.join(col_agg) for col_agg in fb_agg.columns]
fb_agg.head()

,o_p_e_n___m_e_a_n,h_i_g_h___m_i_n,h_i_g_h___m_a_x,l_o_w___m_i_n,l_o_w___m_a_x,c_l_o_s_e___m_e_a_n
trading_volume,,,,,,
low,171.36,129.74,216.20,123.02,212.60,171.43
med,175.82,162.85,218.62,150.75,214.27,175.14
high,167.73,161.10,180.13,149.02,173.75,168.16


In [57]:
weather.loc['2018-10'].query('datatype=="PRCP"').groupby(level=0).mean().head().squeeze()

date
2018-10-01    0.01
2018-10-02    2.23
2018-10-03   19.69
2018-10-04    0.32
2018-10-05    0.97
Name: value, dtype: float64

In [58]:
weather.query('datatype=="PRCP"').groupby(['station_name',pd.Grouper(freq='Q')]).sum().unstack().sample(5,random_state=1)

value                                 
date                        2018-03-31 2018-06-30 2018-09-30 2018-12-31
station_name                                                           
WANTAGH 1.1 NNE, NY US          279.90     216.80     472.50     277.20
STATEN ISLAND 1.4 SE, NY US     379.40     295.30     438.80     409.90
SYOSSET 2.0 SSW, NY US          323.50     263.30     355.50     459.90
STAMFORD 4.2 S, CT US           338.00     272.10     424.70     390.00
WAYNE TWP 0.8 SSW, NJ US        246.20     295.30     620.90     422.00

In [59]:
weather.query('datatype=="PRCP"').groupby(level=0).mean().groupby(pd.Grouper(freq='M')).sum().value.nlargest()

date
2018-11-30   210.59
2018-09-30   193.09
2018-08-31   192.45
2018-07-31   160.98
2018-02-28   158.11
Name: value, dtype: float64

In [61]:
weather.query('datatype=="PRCP"').rename(dict(value='prcp'),axis=1).groupby(level=0).mean().groupby(pd.Grouper(freq='M')).transform(np.sum)['2018-01-28':'2018-02-03']

,prcp
date,
2018-01-28,69.31
2018-01-29,69.31
2018-01-30,69.31
2018-01-31,69.31
2018-02-01,158.11
2018-02-02,158.11
2018-02-03,158.11


In [63]:
weather.query('datatype=="PRCP"').rename(dict(value='prcp'),axis=1).groupby(level=0).mean().assign(
    total_prcp_in_month = lambda x:x.groupby(
        pd.Grouper(freq='M')
    ).transform(np.sum),
    pct_monthly_prcp=lambda x:x.prcp.div(x.total_prcp_in_month)
).nlargest(5,'pct_monthly_prcp')

,prcp,total_prcp_in_month,pct_monthly_prcp
date,,,
2018-10-12,34.77,105.63,0.33
2018-01-13,21.66,69.31,0.31
2018-03-02,38.77,137.46,0.28
2018-04-16,39.34,140.57,0.28
2018-04-17,37.30,140.57,0.27
